This material is copied from [Joris Van den Bossche's Pandas Tutorial](https://github.com/jorisvandenbossche/pandas-tutorial). The material has been heavily edited and added to by Cody Kingham.

# Chapter 13 – Introduction to Pandas

**If you have any questions, drom em in the Slack!**

### Pandas: Data Analysis in Python


Welcome to the chapters on Pandas, a really powerful Python package that enables data structures that are crucial to doing data science, especially DataFrames (i.e. matrices or tables). The R programming language comes with these data structures built-in. But we can essentially do all of the same things in Python with Pandas. Pandas is widely used in the sciences. For example, the recent [first image of a black hole](https://www.jpl.nasa.gov/edu/news/2019/4/19/how-scientists-captured-the-first-image-of-a-black-hole/) was processed using a lot of data from telescopes around the world. That data had to be processed and analyzed to produce the image below. 

<img src="https://www.jpl.nasa.gov/images/universe/20190410/blackhole20190410.jpg" height="20%" width="20%">

Pandas (and numpy, an underlying framework of Pandas) was used in processing this data, as you can see in their publicly available [Github repository](https://github.com/achael/eht-imaging/blob/master/ehtim/statistics/dataframes.py). 

Pandas is also crucial for quantitative and computational linguistics. A wide range of statistical methods rely on data arranged in rows and columns. We will explore more of the linguistic side of things in the assignment for these chapters, as well as in the later chapter specifically dedicated to quantitative linguistic methods. 

## An Ecosystem Unto Itself

In some ways, the Pandas package is an ecosystem unto itself, meaning that it possess a whole range of methods and functions unique to the package. That is common for Python packages. It also means that you **should not expect to master all of Pandas in a few short chapters.** It's important you know how to create basic Pandas objects, and how to do basic maneuvers and interactions. 

## Keep the Pandas documentation on "speed dial": 
## https://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html

The documentation has recently been updated and is very user-friendly. When you get stuck, you can always refer back to it.

## When do you need pandas?

When working with **tabular or structured data** (like R dataframe, SQL table, Excel spreadsheet, ...):

- Import data
- Clean up messy data
- Explore data, gain insight into data
- Process and prepare your data for analysis
- Analyse your data (together with scikit-learn, statsmodels, matplotlib...)

## Why Datascience and Linguistics?

Here's some things we can do with `Pandas` in Python applying some data science methods:

* **build and manipulate large datasets of word or construction frequencies**
* **easily apply statistical measures to linguistic data such as:**
    * efficiently calculate proportions 
    * apply measures of statistical significance between two constructions
    * do multi-variate analysis that analyzes thousands of dimensions for a dataset
* **study word/text meaning using semantic vector spaces**
* **research the interation between various linguistic objects of entities in a text**
* **do authorship analysis by comparing large feature spaces of documents**
* **study how grams change over time by observing their contextual features across corpora from a range of dates**

<hr>

# Demos


**Below we load the Pandas package, as well as [numpy](https://numpy.org), an underlying data structure to Pandas. We also load [Matplotlib](https://matplotlib.org), which we can use to plot data from Pandas.**



In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.options.display.max_rows = 8

Note above that we have imported `pandas` as `pd`. We'll talk more about import statements later. But for now just notice that everywhere you see `pd` it is referring back to the Pandas package itself, along with all of its methods.

**You can use `dir` just like you can with strings and containers to see what can be done with `pd`. You'll see that there are a lot of options to pick from! The Pandas documentation cited above will tell you about each one. But you can also use help to find out what they do.**

In [ ]:
dir(pd)

**Let's look at what `read_csv` does.**

In [ ]:
help(pd.read_csv)

### Case 1: titanic survival data

Below we load a [`.csv` file](https://datahub.io/docs/data-packages/csv) using the Pandas method. You can have a peak at the file under the "data" folder of this repository.

In [ ]:
df = pd.read_csv("../data/titanic.csv") 

Remember that in Python, *everything* is an object. We've already learned about `str`, `bool`, `list`, `set`, and `dict` objects. What kind of object is `df`?

In [ ]:
print(type(df))

We see that this object is a `class` object which comes from the pandas package. It is called a `DataFrame`. We'll talk more about `DataFrames` below. But for now, let's look at what it is and what it can do.

**Below we peak at the top 5 rows in the `DataFrame`. From this view you can see that a `DataFrame` is a kind of table consisting of rows and columns.** This particular dataset contains information about all of the passengers of the Titanic.

In [ ]:
df.head() # show the top 5 rows of df

Starting from reading this dataset, to answering questions about this data in a few lines of code:

**What is the age distribution of the Titanic passengers?**

In [ ]:
df['Age'].hist()

**How does the survival rate of the passengers differ between sexes?**

In [ ]:
df.groupby('Sex')[['Survived']].aggregate(lambda x: x.sum() / len(x))

**Or how does it differ between the different classes?**

In [ ]:
df.groupby('Pclass')['Survived'].aggregate(lambda x: x.sum() / len(x)).plot(kind='bar')

All the needed functionality for the above examples will be explained throughout this tutorial.

### Case 2: air quality measurement timeseries

AirBase (The European Air quality dataBase): hourly measurements of all air quality monitoring stations from Europe

Starting from these hourly data for different stations:

In [ ]:
data = pd.read_csv('../data/20000101_20161231-NO2.csv', sep=';', skiprows=[1], na_values=['n/d'], index_col=0, parse_dates=True)

In [ ]:
data.head()

to answering questions about this data in a few lines of code:

**Does the air pollution show a decreasing trend over the years?**

In [ ]:
data['1999':].resample('M').mean().plot(ylim=[0,120])

In [ ]:
data['1999':].resample('A').mean().plot(ylim=[0,100])

**What is the difference in diurnal profile between weekdays and weekend?**

In [ ]:
data['weekday'] = data.index.weekday
data['weekend'] = data['weekday'].isin([5, 6])
data_weekend = data.groupby(['weekend', data.index.hour])['BASCH'].mean().unstack(level=0)
data_weekend.plot()

We will come back to these example, and build them up step by step.

# 2. The pandas data structures: `DataFrame` and `Series`

A `DataFrame` is a **tablular data structure** (multi-dimensional object to hold labeled data) comprised of rows and columns, akin to a spreadsheet, database table, or R's data.frame object. You can think of it as multiple Series object which share the same index.


<img align="left" width=30% src="../images/dataframe.png">

In Jupyter notebooks, DataFrames are stylized with HTML to make them pretty. We can look at a DataFrame by running a cell. It will show us a subset of the entire DataFrame. The full dimensions are displayed below the table.

In [ ]:
df

## DataFrame

The `DataFrame` is a kind of container object in a similar way as a `dictionary`. 

### Attributes of the DataFrame

Like dictionaries have `keys` and `values`, the contents of a DataFrame can be accessed using a set of attributes. The primary attributes for `DataFrames` are as follows:

In [ ]:
df.index # row labels

Note that `index` in Pandas DataFrames refers to **rows**. The row labels above are continuous, 0-891, and thus summarized. That is different from the column labels as you'll see below.

In [ ]:
df.columns # column labels

The `values` attribute returns an array (a `numpy` version of lists) or arrays that contains the whole dataset.

In [ ]:
df.values

The `shape` attribute is very useful for getting a sense of the size of the dataset. The format is `(n_rows, n_columns)`.

In [ ]:
df.shape

DataFrames can contain all kinds of different object types. Standard Python objects like `int` or `str` are put into object types conducive for DataFrames. To check the data types of the different columns:

In [ ]:
df.dtypes

An overview of that information can be given with the `info()` method:

In [ ]:
df.info()

### Building DataFrames from Scratch.

#### With a list

Apart from importing your data from an external source (text file, excel, database, ..), one of the most common ways of creating a dataframe is from a dictionary of arrays or lists.

Note that with this method, each list represents a single observation or, in this case, a country. You could use other ordered objects as well, such as `tuples`: i.e. tuple of lists, list of tuples, tuple of tuples, list of lists.

In [ ]:
data = [
    ['Belgium', 11.3, 30510, 'Brussels'],
    ['France', 64.3, 671308, 'Paris'],
    ['Germany', 81.3, 357050, 'Berlin'],
    ['Netherlands', 16.9, 41526, 'Amsterdam'],
    ['United Kingdom', 64.9, 244820, 'London']
]

headers = ['country', 'population', 'area', 'capital'] # column headers

df_countries = pd.DataFrame(data, columns=headers) # what happens if we don't say `columns=headers`?
df_countries

#### With a dictionary

Rather than feeding a list of rows, we can provide a dictionary of columns. In this case, the keys of the dictionary are the column labels and the values are some kind of ordered iterable (e.g. `list`, `tuple`, `pandas.Series`). Note that the items for each iterable should be in the order of the other iterables. For example, $64.3$ in "population" should correspond with "France" in 'country'.

In [ ]:
data = {
    'country': ['Belgium', 'France', 'Germany', 'Netherlands', 'United Kingdom'],
    'population': [11.3, 64.3, 81.3, 16.9, 64.9],
    'area': [30510, 671308, 357050, 41526, 244820],
    'capital': ['Brussels', 'Paris', 'Berlin', 'Amsterdam', 'London']
}
df_countries = pd.DataFrame(data)
df_countries

## Thinking in terms of "vectorization"

In Python, when we have a lot of items that we want to alter we typically put them into a container, loop over the container, and do something to the items. This method of altering values on a one-by-one basis is too inefficient and time consuming for data analysis. When working with Pandas, you need to think in terms of operations applied to entire datasets **at once**. What do I mean? 

Have a look at the case below.

In [ ]:
freq_dict = {
    'the': 10241,
    'you': 9023,
    'run': 4048,
    'get': 123,
}

 We have a frequency dictionary which contains counts for tokens in a corpus. If we want to convert these values to a proportion of the total, in Python we have to use a loop.

In [ ]:
prop_dict = {}

freq_dict_total = sum(freq_dict.values())

for token, count in freq_dict.items():
    prop = round(count / freq_dict_total, 2)
    prop_dict[token] = prop
    
prop_dict

If we have a very large dataset (100,000s of items) and need to do more complex transformations than proportions, this would get very inefficient. 

`Pandas` relies on a concept called ["vectorization"](https://www.quantifisolutions.com/vectorization-part-2-why-and-what) by which we can operate on an entire dataset at once. This method is used heavily in the programming language R, for instance.

In [ ]:
tokens = ['the', 'you', 'run', 'get']
freqs = [10241, 9023, 4048, 123]
freq_df = pd.DataFrame(freqs, index=tokens, columns=['freq'])
freq_df

We can transform the values in the `freq` column of this dataframe into proportions with a single line of code.

In [ ]:
freq_df / freq_df.sum()

The vectorized methods have changed all of the contained data simultaneously rather than item-by-item. This is the power of Pandas.

### One-dimensional data: `Series` (a column of a DataFrame)

A Series is a basic holder for **one-dimensional labeled data**, similar to a `list` or `tuple`, but possessing special methods and optimizations for data analysis. A `DataFrame` consists of `Series` objects "glued" together. For instance, if we select the "Age" column below we'll see that the column *is* a `Series`.

In [ ]:
age = df['Age']
print(type(df['Age']))

In [ ]:
age

The `Series` has a lot of useful methods. Here are some examples:

In [ ]:
print('mean', age.mean())
print('max', age.max())
print('min', age.min())
print('mean', age.mean())
print('sum', age.sum())
print('quantile', age.quantile(1))

### Attributes of a Series: `index` and `values`

The Series has also an `index` and `values` attribute, but no `columns`

In [ ]:
age.index

You can access the underlying numpy array representation with the `.values` attribute:

In [ ]:
age.values[:10]

We can access series values via the index, just like for NumPy arrays:

In [ ]:
age[0]

Unlike the NumPy array, though, this index can be something other than integers:

In [ ]:
df = df.set_index('Name')
df

In [ ]:
age = df['Age']
age

In [ ]:
age['Dooley, Mr. Patrick']

but with the power of numpy arrays. Many things you can do with numpy arrays, can also be applied on DataFrames / Series.

Eg element-wise operations:

In [ ]:
age * 1000

A range of methods:

In [ ]:
age.mean()

Fancy indexing, like indexing with a list or boolean indexing:

In [ ]:
age[age > 70]

But also a lot of pandas specific methods, e.g.

In [ ]:
df['Embarked'].value_counts()

<div class="alert alert-success">

<b>EXERCISE</b>:

 <ul>
  <li>What is the maximum Fare that was paid? And the median?</li>
</ul>
</div>

<div class="alert alert-success">

<b>EXERCISE</b>:

 <ul>
  <li>Calculate the average survival ratio for all passengers (note: the 'Survived' column indicates whether someone survived (1) or not (0)).</li>
</ul>
</div>

# 3. Data import and export

A wide range of input/output formats are natively supported by pandas:

* CSV, text
* SQL database
* Excel
* HDF5
* json
* html
* pickle
* sas, stata
* (parquet)
* ...

In [ ]:
#pd.read

In [ ]:
#df.to

Very powerful csv reader:

In [ ]:
pd.read_csv?

Luckily, if we have a well formed csv file, we don't need many of those arguments:

In [ ]:
df = pd.read_csv("../data/titanic.csv")

In [ ]:
df.head()

<div class="alert alert-success">

<b>EXERCISE</b>: Read the `data/20000101_20161231-NO2.csv` file into a DataFrame `no2`
<br><br>
Some aspects about the file:
 <ul>
  <li>Which separator is used in the file?</li>
  <li>The second row includes unit information and should be skipped (check `skiprows` keyword)</li>
  <li>For missing values, it uses the `'n/d'` notation (check `na_values` keyword)</li>
  <li>We want to parse the 'timestamp' column as datetimes (check the `parse_dates` keyword)</li>
</ul>
</div>

# 4. Exploration

Some useful methods:

`head` and `tail`

In [ ]:
no2.head(3)

In [ ]:
no2.tail()

`info()`

In [ ]:
no2.info()

Getting some basic summary statistics about the data with `describe`:

In [ ]:
no2.describe()

Quickly visualizing the data

In [ ]:
no2.plot(kind='box', ylim=[0,250])

In [ ]:
no2['BASCH'].plot(kind='hist', bins=50)

<div class="alert alert-success">

<b>EXERCISE</b>: 

 <ul>
  <li>Plot the age distribution of the titanic passengers</li>
</ul>
</div>

The default plot (when not specifying `kind`) is a line plot of all columns:

In [ ]:
no2.plot(figsize=(12,6))

This does not say too much ..

We can select part of the data (eg the latest 500 data points):

In [ ]:
no2[-500:].plot(figsize=(12,6))

Or we can use some more advanced time series features -> see further in this notebook!

# 5. Selecting and filtering data

<div class="alert alert-warning">
<b>ATTENTION!</b>: <br><br>

One of pandas' basic features is the labeling of rows and columns, but this makes indexing also a bit more complex compared to numpy. <br><br> We now have to distuinguish between:

 <ul>
  <li>selection by **label**</li>
  <li>selection by **position**</li>
</ul>
</div>

In [ ]:
df = pd.read_csv("../data/titanic.csv")

### `df[]` provides some convenience shortcuts 

For a DataFrame, basic indexing selects the columns.

Selecting a single column:

In [ ]:
df['Age']

or multiple columns:

In [ ]:
df[['Age', 'Fare']]

But, slicing accesses the rows:

In [ ]:
df[10:15]

### Systematic indexing with `loc` and `iloc`

When using `[]` like above, you can only select from one axis at once (rows or columns, not both). For more advanced indexing, you have some extra attributes:
    
* `loc`: selection by label
* `iloc`: selection by position

In [ ]:
df = df.set_index('Name')

Note that we've now changed the index to the `Name` column:

In [ ]:
df.head()

In [ ]:
df.loc['Bonnell, Miss. Elizabeth', 'Fare']

In [ ]:
df.loc['Bonnell, Miss. Elizabeth':'Andersson, Mr. Anders Johan', :]

Selecting by position with `iloc` works similar as indexing numpy arrays:

In [ ]:
df.iloc[0:2,1:3]

The different indexing methods can also be used to assign data:

In [ ]:
df.loc['Braund, Mr. Owen Harris', 'Survived'] = 100

In [ ]:
df

### Boolean indexing (filtering)

Often, you want to select rows based on a certain condition. This can be done with 'boolean indexing' (like a where clause in SQL) and comparable to numpy. 

The indexer (or boolean mask) should be 1-dimensional and the same length as the thing being indexed.

In [ ]:
df['Fare'] > 50

In [ ]:
df[df['Fare'] > 50]

<div class="alert alert-success">

<b>EXERCISE</b>:

 <ul>
  <li>Based on the titanic data set, select all rows for male passengers and calculate the mean age of those passengers. Do the same for the female passengers</li>
</ul>
</div>

In [ ]:
df = pd.read_csv("../data/titanic.csv")

<div class="alert alert-success">

<b>EXERCISE</b>:

 <ul>
  <li>Based on the titanic data set, how many passengers older than 70 were on the Titanic?</li>
</ul>
</div>

# 6. The group-by operation

### Some 'theory': the groupby operation (split-apply-combine)

In [ ]:
df = pd.DataFrame({'key':['A','B','C','A','B','C','A','B','C'],
                   'data': [0, 5, 10, 5, 10, 15, 10, 15, 20]})
df

### Recap: aggregating functions

When analyzing data, you often calculate summary statistics (aggregations like the mean, max, ...). As we have seen before, we can easily calculate such a statistic for a Series or column using one of the many available methods. For example:

In [ ]:
df['data'].sum()

However, in many cases your data has certain groups in it, and in that case, you may want to calculate this statistic for each of the groups.

For example, in the above dataframe `df`, there is a column 'key' which has three possible values: 'A', 'B' and 'C'. When we want to calculate the sum for each of those groups, we could do the following:

In [ ]:
for key in ['A', 'B', 'C']:
    print(key, df[df['key'] == key]['data'].sum())

This becomes very verbose when having multiple groups. You could make the above a bit easier by looping over the different values, but still, it is not very convenient to work with.

What we did above, applying a function on different groups, is a "groupby operation", and pandas provides some convenient functionality for this.

### Groupby: applying functions per group

The "group by" concept: we want to **apply the same function on subsets of your dataframe, based on some key to split the dataframe in subsets**

This operation is also referred to as the "split-apply-combine" operation, involving the following steps:

* **Splitting** the data into groups based on some criteria
* **Applying** a function to each group independently
* **Combining** the results into a data structure

<img src="../images/splitApplyCombine.png">

Similar to SQL `GROUP BY`

Instead of doing the manual filtering as above


    df[df['key'] == "A"].sum()
    df[df['key'] == "B"].sum()
    ...

pandas provides the `groupby` method to do exactly this:

In [ ]:
df.groupby('key').sum()

In [ ]:
df.groupby('key').aggregate(np.sum)  # 'sum'

Have a look at the object type that `df.groupby` yields:

In [ ]:
type(df.groupby('key'))

We can index this object similarly to a DataFrame. For instance, to sum on a particular column, returning a Series:

In [ ]:
df.groupby('key')['data'].sum()

Or, to keep it a DataFrame, pass a list of column names into the index brackets `[ ['some', 'columns',] ]`:

In [ ]:
df.groupby('key')[['data']].sum()

### Application of the groupby concept on the titanic data

We go back to the titanic passengers survival data:

In [ ]:
df = pd.read_csv("../data/titanic.csv")

In [ ]:
df.head()

<div class="alert alert-success">

<b>EXERCISE</b>:

 <ul>
  <li>Calculate the average age for each sex again, but now using groupby.</li>
</ul>
</div>

<div class="alert alert-success">

<b>EXERCISE</b>:

 <ul>
  <li>Calculate the average survival ratio for all passengers.</li>
</ul>
</div>

<div class="alert alert-success">

<b>EXERCISE</b>:

 <ul>
  <li>Calculate this survival ratio for all passengers 25 and younger (remember: filtering/boolean indexing).</li>
</ul>
</div>

<div class="alert alert-success">

<b>EXERCISE</b>:

 <ul>
  <li>What is the difference in the survival ratio between the sexes?</li>
</ul>
</div>

<div class="alert alert-success">

<b>EXERCISE</b>:

 <ul>
  <li>Or how does it differ between the different classes? Make a bar plot visualizing the survival ratio for the 3 classes.</li>
</ul>
</div>

In data science we make a distinction between categorical and continuous variables. A categorical varible is, as it sounds, a discrete category. For instance "apple", "orange", and "pear" are all categories of fruit, categorical data. An example of a continuous variable would be a measurement like speed, where values can range along the number line. 

What if we want to convert a continuous variable, such as Age, into categories, that is, age ranges? Have a look at the `pd.cut` method below.

In [ ]:
help(pd.cut)

An example of this in action is given below.

In [ ]:
df['AgeClass'] = pd.cut(df['Age'], bins=np.arange(0,90,10))

Don't worry if you don't understand everything here yet. Note that `np.arange`, a `numpy` method has created an empty matrix that ranges from 0 to 90 in steps of 10. These serve as "bins" into which the new data is organized. Play around with `np.arange` in a new cell if you don't quite understand. Although we don't explicitly discuss `numpy` methods, it is helpful to at least be aware of its existence. 

Notice that we've added a new column using this method. Let's have a look at it:

In [ ]:
df['AgeClass']

Note that each of the values in the column is a "bin", that is, it is simply a label explaining which group each row belongs to. We can use `groupby` to organize the data on this group just as we can with any other group.

<div class="alert alert-success">

<b>EXERCISE</b>:

 <ul>
  <li>Make a bar plot to visualize the average Fare payed by people depending on their AgeClass.</li>
</ul>
</div>

# 7. Working with time series data

Datasets often contain times or dates, which are often used as independent variables in a plot or analysis. But with standard Python objects, dates would be difficult to work with: strings sort based on alphabetical order not time values, and integers do not transparently represent time. For this purpose, Pandas has a series of objects and methods that can be used to analyze continuous ranges of dates or times. 

Pay attention to the keyword argument provided below as we load the `.csv` data: `parse_dates`.

In [ ]:
no2 = pd.read_csv('../data/20000101_20161231-NO2.csv', sep=';', skiprows=[1], na_values=['n/d'], index_col=0, parse_dates=True)

In [ ]:
no2

When we ensure the DataFrame has a `DatetimeIndex`, time-series related functionality becomes available:

In [ ]:
no2.index

Indexing a time series works with strings:

In [ ]:
no2["2010-01-01 09:00": "2010-01-01 12:00"]

A nice feature is "partial string" indexing, so you don't need to provide the full datetime string.

E.g. all data of January up to March 2012:

In [ ]:
no2['2012-01':'2012-03']

Time and date components can be accessed from the index:

In [ ]:
no2.index.hour

In [ ]:
no2.index.year

## Converting your time series with `resample`

A very powerfull method is **`resample`: converting the frequency of the time series** (e.g. from hourly to daily data).

Remember the air quality data:

In [ ]:
no2.plot()

The time series has a frequency of 1 hour. I want to change this to daily:

In [ ]:
no2.head()

In [ ]:
no2.resample('D').mean().head()

Above I take the mean, but as with `groupby` I can also specify other methods:

In [ ]:
no2.resample('D').max().head()

The string to specify the new time frequency: http://pandas.pydata.org/pandas-docs/dev/timeseries.html#offset-aliases  
These strings can also be combined with numbers, eg `'10D'`.

Further exploring the data:

In [ ]:
no2.resample('M').mean().plot() # 'A'

In [ ]:
# no2['2012'].resample('D').plot()

<div class="alert alert-success">

<b>EXERCISE</b>: The evolution of the yearly averages with, and the overall mean of all stations

 <ul>
  <li>Use `resample` and `plot` to plot the yearly averages for the different stations.</li>
  <li>The overall mean of all stations can be calculated by taking the mean of the different columns (`.mean(axis=1)`).</li>
</ul>
</div>

<div class="alert alert-success">

<b>EXERCISE</b>: how does the *typical monthly profile* look like for the different stations?

 <ul>
  <li>Add a 'month' column to the dataframe.</li>
  <li>Group by the month to obtain the typical monthly averages over the different years.</li>
</ul>
</div>

First, we add a column to the dataframe that indicates the month (integer value of 1 to 12):

Now, we can calculate the mean of each month over the different years:

<div class="alert alert-success">

<b>EXERCISE</b>: The typical diurnal profile for the different stations

 <ul>
  <li>Similar as for the month, you can now group by the hour of the day.</li>
</ul>
</div>

<div class="alert alert-success">

<b>EXERCISE</b>: What is the difference in the typical diurnal profile between week and weekend days for the 'BASCH' station.

 <ul>
  <li>Add a column 'weekday' defining the different days in the week.</li>
  <li>Add a column 'weekend' defining if a days is in the weekend (i.e. days 5 and 6) or not (True/False).</li>
  <li>You can groupby on multiple items at the same time. In this case you would need to group by both weekend/weekday and hour of the day.</li>
</ul>
</div>

Add a column indicating the weekday:

In [ ]:
no2.index.weekday?

Add a column indicating week/weekend

Now we can groupby the hour of the day and the weekend (or use `pivot_table`):

<div class="alert alert-success">

<b>EXERCISE</b>: What are the number of exceedances of hourly values above the European limit 200 µg/m3 ?

Count the number of exceedances of hourly values above the European limit 200 µg/m3 for each year and station after 2005. Make a barplot of the counts. Add an horizontal line indicating the maximum number of exceedances (which is 18) allowed per year?
<br><br>

Hints:

 <ul>
  <li>Create a new DataFrame, called `exceedances`, (with boolean values) indicating if the threshold is exceeded or not</li>
  <li>Remember that the sum of True values can be used to count elements. Do this using groupby for each year.</li>
  <li>Adding a horizontal line can be done with the matplotlib function `ax.axhline`.</li>
</ul>
</div>

In [ ]:
# re-reading the data to have a clean version
no2 = pd.read_csv('../data/20000101_20161231-NO2.csv', sep=';', skiprows=[1], na_values=['n/d'], index_col=0, parse_dates=True)

# 9. What I didn't talk about

- Concatenating data: `pd.concat`
- Merging and joining data: `pd.merge`
- Reshaping data: `pivot_table`, `melt`, `stack`, `unstack`
- Working with missing data: `isnull`, `dropna`, `interpolate`, ...
- ...


## Further reading

* Pandas documentation: http://pandas.pydata.org/pandas-docs/stable/

* Books

    * "Python for Data Analysis" by Wes McKinney
    * "Python Data Science Handbook" by Jake VanderPlas

* Tutorials (many good online tutorials!)

  * https://github.com/jorisvandenbossche/pandas-tutorial
  * https://github.com/brandon-rhodes/pycon-pandas-tutorial

* Tom Augspurger's blog

  * https://tomaugspurger.github.io/modern-1.html